In [1]:
import wtss
import pandas as pd
import geopandas as gpd

import numpy as np

import tqdm

In [2]:
rng = np.random.default_rng(20241217)

In [3]:
base_dir = '/home/luiz_gabriel/projects/estimava_intensificacao_vertical_irrigacao/data/'
pontos_path = base_dir + 'grades_com_id/grades_com_id_do_pivo.shp'

In [4]:
pontos = gpd.read_file(pontos_path)

In [5]:
pontos.head()

,left,top,right,bottom,Hectares,din_class,din_classF,id_2,id_ponto,geometry
0,-47.378595,-16.784793,-47.377925,-16.784123,75.606338,WC,SAFRA DUPLA (INVERNO),0,0,POINT (-47.3786 -16.78479)
1,-47.378595,-16.785463,-47.377925,-16.784793,75.606338,WC,SAFRA DUPLA (INVERNO),0,1,POINT (-47.3786 -16.78546)
2,-47.378595,-16.786133,-47.377925,-16.785463,75.606338,WC,SAFRA DUPLA (INVERNO),0,2,POINT (-47.3786 -16.78613)
3,-47.378595,-16.786803,-47.377925,-16.786133,75.606338,WC,SAFRA DUPLA (INVERNO),0,3,POINT (-47.3786 -16.7868)
4,-47.378595,-16.787473,-47.377925,-16.786803,75.606338,WC,SAFRA DUPLA (INVERNO),0,4,POINT (-47.3786 -16.78747)


In [6]:
pontos = pontos.groupby('id_2', group_keys=False).apply(lambda x: x.sample(
    frac=0.20, random_state=rng))

/tmp/ipykernel_13898/2341409223.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pontos = pontos.groupby('id_2', group_keys=False).apply(lambda x: x.sample(


In [7]:
servico = wtss.WTSS('https://data.inpe.br/bdc/wtss/v4/')
servico.coverages

['CBERS4-MUX-2M-1',
 'CBERS4-WFI-16D-2',
 'CBERS-WFI-8D-1',
 'LANDSAT-16D-1',
 'mod13q1-6.1',
 'myd13q1-6.1',
 'S2-16D-2']

In [8]:
cubo_s2 = servico['S2-16D-2']

In [9]:
timeline = cubo_s2.timeline

ultima_data_disponivel = timeline[0]
primeira_data_disponivel = timeline[-1]

print(f'Intervalo: [{primeira_data_disponivel}, {ultima_data_disponivel}]')

Intervalo: [2024-07-27, 2017-01-01]


In [10]:
start_date = '2017-06-01'
end_date = '2022-12-31'

attributes = ('NDVI', 'SCL')

In [11]:


ndvi_values = []
scl_values = []

for id, geometry in tqdm.tqdm(zip(pontos.id_ponto.values,
                                  pontos.geometry.values),
                         desc="Processando dados", unit="data"):

    timeseries = cubo_s2.ts(attributes=attributes,
                            geom=geometry,
                            start_date=start_date,
                            end_date=end_date)
    
    tmp = timeseries.df('NDVI')   

    ndvi_values.append(tmp.loc[tmp.attribute == 'NDVI', 'value'])
    scl_values.append(tmp.loc[tmp.attribute == 'SCL', 'value'])



Processando dados: 0data [00:00, ?data/s]

Processando dados: 23260data [5:21:40,  1.21data/s]


In [ ]:
ndvi_values

array([    47,    113,    132, ..., 116282, 116288, 116293])

In [17]:
ndvi = pd.DataFrame(data = np.asarray(ndvi_values).T,
                    columns=pontos.id_ponto.values,
                    index=timeseries.ts.timeline)

In [19]:
ndvi = ndvi.reset_index().rename(columns={'index': 'data'})

In [21]:
ndvi.to_csv(base_dir + 'ndvi_sample_v2.csv')

In [22]:
scl = pd.DataFrame(data = np.asarray(scl_values).T,
                    columns=pontos.id_ponto.values,
                    index=timeseries.ts.timeline)

scl = scl.reset_index().rename(columns={'index': 'data'})


In [24]:
scl.to_csv(base_dir + 'scl_sample_v2.csv')